## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd
import glob
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import requests
import time

## Data Read-in

In [2]:
csv_files = glob.glob('*.csv')

df_list = []

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    df_list.append(df)
    
df = pd.concat(df_list,ignore_index=True)

print(len(df))

2753


In [3]:
gdf = gpd.read_file('countyboundary')

## Previous month's values

Get values from here:

https://docs.google.com/spreadsheets/d/1GWlqTIzCB29pDVDjd-iBVejVGGsLiAd0P6fq4LM83WI/edit?usp=sharing

In [4]:
print('Input County Name:')
county_name = input()

Input County Name:
Miami-Dade County


In [5]:
print('Input total condo units sold previous (not last) month:')
total_condos_sold_2nd_month = input()

Input total condo units sold previous (not last) month:
712


In [6]:
print('Input total sales volume ($$$ million) from previous (not last) month:')
sales_volume_2nd_month = input()

Input total sales volume ($$$ million) from previous (not last) month:
470_000_000


In [7]:
print('Input median sales price from previous (not last) month:')
median_sales_price_2nd_month = input()

Input median sales price from previous (not last) month:
380_000


In [16]:
print('Input median psf from previous (not last) month:')
median_psf_2nd_month = input()

Input median psf from previous (not last) month:
388


## Data Clean

In [17]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [18]:
df = df.dropna(subset=['SOLD DATE'])

In [19]:
# Get the current date
current_date = datetime.now()

# Calculate the first day of the current month
first_day_of_current_month = current_date.replace(day=1)

# Get the last day of the previous month by subtracting one day from the first day of the current month
last_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the month name of the previous month
previous_month_name = last_day_of_previous_month.strftime("%B")

# ---- Get the name of the month, as a string, from two months ago.
# ---- Meaning, if the current month is January, return November.

# Get the first day of the previous month by subtracting one day from the first day of the current month
first_day_of_previous_month = first_day_of_current_month - timedelta(days=1)

# Get the first day of the month before the last by subtracting one day from the first day of the previous month
first_day_of_month_before_last = first_day_of_previous_month.replace(day=1) - timedelta(days=1)

# Get the month name of the month before the last
month_before_last_name = first_day_of_month_before_last.strftime("%B")

print(f'Report month: {previous_month_name} ---- Month before report month: {month_before_last_name}')

Report month: September ---- Month before report month: August


In [20]:
# Define list of desired months (excluding current month)
desired_months = [previous_month_name]

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [21]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    666
Name: count, dtype: int64
-------
$/SQUARE FEET
False    664
True       2
Name: count, dtype: int64
-------
YEAR BUILT
False    666
Name: count, dtype: int64
-------


In [22]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Miami/300-Biscayne-Boulevard-Way-33131/unit-1706/home/191553278


In [23]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [24]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [25]:
df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)

In [26]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
177,PAST SALE,September-25-2024,Condo/Co-op,19201 Collins Ave #938,Sunny Isles Beach,FL,33160.0,118.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/19...,MARMLS,A11595355,N,Y,25.954318,-80.119900
146,PAST SALE,September-30-2024,Condo/Co-op,7636 Abbott Ave #6,Miami Beach,FL,33141.0,170.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/7636-Abb...,MARMLS,A11636168,N,Y,25.861882,-80.123252
189,PAST SALE,September-10-2024,Condo/Co-op,1750 NE 191st St Unit 209-2,Miami,FL,33179.0,16500.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1750-NE-191st-...,MARMLS,A11481701,N,Y,25.949213,-80.167410
188,PAST SALE,September-6-2024,Condo/Co-op,3255 NE 184th St #12105,Aventura,FL,33160.0,49000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/32...,Beaches MLS,F10423705,N,Y,25.947065,-80.137548
159,PAST SALE,September-12-2024,Condo/Co-op,15201 NE 6th Ave Unit C210,Miami,FL,33162.0,86000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15201-NE-6th-A...,MARMLS,A11609102,N,Y,25.915314,-80.186602
144,PAST SALE,September-6-2024,Condo/Co-op,101 NW 204th St #5,Miami Gardens,FL,33169.0,99000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/101-NW-204th-S...,MARMLS,A11623245,N,Y,25.963244,-80.204667
162,PAST SALE,September-10-2024,Condo/Co-op,15610 NE 6th Ave Unit 30C,Miami,FL,33162.0,110000.0,2.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/15610-NE-6th-A...,Beaches MLS,F10450532,N,Y,25.918155,-80.188500
191,PAST SALE,September-6-2024,Condo/Co-op,51 NE 204th St #19,Miami Gardens,FL,33179.0,110000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/51-NE-204th-St...,MARMLS,A11509500,N,Y,25.963309,-80.199903
153,PAST SALE,September-27-2024,Condo/Co-op,498 NW 165th St Rd Unit D-501,Miami,FL,33169.0,120000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/498-NW-165th-S...,MARMLS,A11624929,N,Y,25.923320,-80.207215
187,PAST SALE,September-11-2024,Condo/Co-op,661 NE 195 #107,Miami,FL,33179.0,120000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/661-NE-195th-S...,Beaches MLS,RX-10966050,N,Y,25.955972,-80.190621


In [27]:
print(df_filtered['URL'].iloc[61])

https://www.redfin.com/FL/Miami/777-NE-62nd-St-33138/unit-C216/home/43306760


In [28]:
# Correct the prices, if needed
df_filtered.at[177,'PRICE']=(118_000)
df_filtered.at[146,'PRICE']=(170_000)
df_filtered.at[189,'PRICE']=(165_000)
df_filtered.at[188,'PRICE']=(490_000)


# Correct the psf, if needed
df_filtered.at[177,'$/SQUARE FEET']=(118_000/360)
df_filtered.at[146,'$/SQUARE FEET']=(170_000/521)
df_filtered.at[189,'$/SQUARE FEET']=(189_000/1_317)
df_filtered.at[188,'$/SQUARE FEET']=(499_000/1_077)

In [61]:
### If needed, drop the row with the lowest price
# min_value_index = df_filtered['PRICE'].idxmin()
# df_filtered = df_filtered.drop(min_value_index)

In [24]:
# print(df_filtered['URL'].iloc[528])

In [29]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
190,195000.0,5300 NW 87th Ave #512,Doral,94.000000
153,120000.0,498 NW 165th St Rd Unit D-501,Miami,116.000000
159,86000.0,15201 NE 6th Ave Unit C210,Miami,116.000000
166,130000.0,1750 NE 191st St Unit 315-3,Miami,125.000000
187,120000.0,661 NE 195 #107,Miami,126.000000
162,110000.0,15610 NE 6th Ave Unit 30C,Miami,126.000000
157,132000.0,19051 NE 2nd Ave #1515,Miami,129.000000
174,129000.0,165 NE 203rd Ter #9,Miami Gardens,138.000000
165,143000.0,484 NW 165th St Rd Unit A216,Miami,138.000000
182,145000.0,1710 NE 191 #316,Miami,140.000000


In [63]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [30]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [31]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [32]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [33]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [34]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [35]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"{previous_month_name} 2024 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

folium.GeoJson(gdf,tooltip=f'{county_name}',name=f'{county_name}').add_to(m)

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
# m

In [36]:
m.save('index.html')

## Summary Info

In [37]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [38]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [39]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [40]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/752-Meridian-Ave-33139/unit-19/home/42802202


In [41]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
OCEANA KEY BISCAYNE CONDO, 360 Ocean Dr Unit 1002S Key Biscayne | Price $8,400,000 | $2,504 psf | Year built: 2014
Least Expensive
OLA CONDO, 15201 NE 6th Ave Unit C210 Miami | Price $86,000 | $116 psf | Year built: 1967
Highest Price Per Square Foot
SURF CLUB CONDO, 9111 Collins Ave Unit N-713 Surfside | Price $7,750,000 | $4,251 psf | Year built: 2017
Lowest Price Per Square Foot
THE BLUE A RESORT HOTEL C, 5300 NW 87th Ave #512 Doral | Price $195,000 | $94 psf | Year built: 2007
Newest
Mr. C Residences, 2655 S Bayshore Dr #1501 Miami | Price $7,000,000 | $2,578 psf | Year built: 2024
Oldest
PALM GARDEN CONDO, 752 Meridian Ave #19 Miami Beach | Price $380,000 | $745 psf | Year built: 1923


## Map URL Snagger

In [42]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDadeCounty_condo_sales_month_ending_sept_2024


## Get Summary Data

In [43]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 626
--------
Total sale price: $438,473,689
--------
Median sale price: $380,000
--------
Max sale price: $8,400,000
--------
Min sale price: $86,000
------------------------------------------------
PSF INFO
Max price per square foot: $4,251
--------
Min price per square foot: $94
--------
Median price per square foot: $401
------------------------------------------------
CONDO AGES
Newest building: 2024.0
----------
Oldest building: 1923.0
----------
Average building age: 1990.0335463258787


## Collect Agent Data

In [44]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [45]:
df_filtered.reset_index(inplace=True,drop=True)

In [46]:
max_price_index = df_filtered['PRICE'].idxmax()
min_price_index = df_filtered['PRICE'].idxmin()

max_price_psf_index = df_filtered['$/SQUARE FEET'].idxmax()
min_price_psf_index = df_filtered['$/SQUARE FEET'].idxmin()

max_year_built_index = df_filtered['YEAR BUILT'].idxmax()
min_year_built_index = df_filtered['YEAR BUILT'].idxmin()

In [47]:
index_list = [max_price_index,min_price_index,
             max_price_psf_index,min_price_psf_index,
             max_year_built_index,min_year_built_index]

In [48]:
response_list = []

for index, row in df_filtered.iterrows():
    if index in index_list:
        response_list.append(process_response(row['URL']))
        
response_df = pd.DataFrame(response_list)

merged_df = pd.merge(left=df_filtered,left_on='URL',right=response_df, right_on='url',how='outer')

In [49]:
### Test to see if data was collected
merged_df['list_agent_name_1'].unique()

array([nan, 'Johanna Viac', 'Giulietta Ulloa', 'Sandra Diaz',
       'Itai Tsanaani', 'Daniel Tzinker', 'Ximena Penuela'], dtype=object)

In [50]:
df_filtered = merged_df

In [51]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")

Most Expensive
OCEANA KEY BISCAYNE CONDO, 360 Ocean Dr Unit 1002S Key Biscayne | Price $8,400,000 | $2,504 psf | Year built: 2014 | Listing agents: Giulietta Ulloa with BHHS EWM Realty and Ximena Ulloa with BHHS EWM Realty | Buyer's agents: Giulietta Ulloa with BHHS EWM Realty and None with None
Least Expensive
OLA CONDO, 15201 NE 6th Ave Unit C210 Miami | Price $86,000 | $116 psf | Year built: 1967 | Listing agents: Itai Tsanaani with Folio360 Group, LLC. and None with None | Buyer's agents: Itai Tsanaani with Folio360 Group, LLC. and None with None
Highest Price Per Square Foot
SURF CLUB CONDO, 9111 Collins Ave Unit N-713 Surfside | Price $7,750,000 | $4,251 psf | Year built: 2017 | Listing agents: Ximena Penuela with Fort Realty, LLC and None with None | Buyer's agents: Sonia Navarro with Miami Exclusive Properties and None with None
Lowest Price Per Square Foot
THE BLUE A RESORT HOTEL C, 5300 NW 87th Ave #512 Doral | Price $195,000 | $94 psf | Year built: 2007 | Listing agents: Joh

## Get condo names and locations for most and least expensive

In [52]:
most_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']
most_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmax()]['CITY']

least_expensive_muni_condo_name = df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']
least_expensive_muni_location = df_filtered.loc[df_filtered['PRICE'].idxmin()]['CITY']

In [53]:
## Calculate the increase/descrease between both months
subject_month_sales_volume = df_filtered["PRICE"].sum()
prior_month_sales_volume = int(sales_volume_2nd_month)  # Example value, replace with actual value

subject_month_closings = len(df_filtered)
prior_month_closings = int(total_condos_sold_2nd_month)

# Subtract the smaller value from the larger one
if subject_month_sales_volume > prior_month_sales_volume:
    hed_rf = 'jumps'
    nut_graf = 'rising'
    second_nut_graf_ref = 'up'
    social = 'higher'
    seo = 'increased'
elif prior_month_sales_volume > subject_month_sales_volume:
    hed_rf = 'drops'
    nut_graf = 'falling'
    second_nut_graf_ref = 'down'
    social = 'lower'
    seo = 'dropped'
else:
    hed_rf = 'equals'
    nut_graf = 'equaling'
    second_nut_graf_ref = 'on par with'
    social = 'equal'
    seo = "did't change"
    
    
if subject_month_closings > prior_month_closings:
    dek_rf = 'rose'
elif prior_month_closings > subject_month_closings:
    dek_rf ='fell'
else:
    dek_rf = 'equaled'

In [54]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK', 'FULL_ADDRESS', 'url',
       'list_agent_name_1', 'list_broker_name_1', 'list_agent_name_2',
       'list_broker_name_2', 'buy_agent_name_1', 'buy_broker_name_1',
       'buy_agent_name_2', 'buy_broker_name_2'],
      dtype='object')

In [56]:
# Get values for the top sale (max price)
top_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_1']
top_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_1']
top_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_agent_name_2']
top_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'list_broker_name_2']
top_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_1']
top_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_1']
top_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_agent_name_2']
top_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmax(), 'buy_broker_name_2']

# Get values for the bottom sale (min price)
bottom_sale_listing_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_1']
bottom_sale_listing_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_1']
bottom_sale_listing_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_agent_name_2']
bottom_sale_listing_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'list_broker_name_2']
bottom_sale_buyer_agent_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_1']
bottom_sale_buyer_broker_1 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_1']
bottom_sale_buyer_agent_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_agent_name_2']
bottom_sale_buyer_broker_2 = df_filtered.loc[df_filtered["PRICE"].idxmin(), 'buy_broker_name_2']

# Now create the formatted string
test_string = f'''


TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

'''

print(test_string)





TOP SALE LISTING AGENTS:

Giulietta Ulloa with BHHS EWM Realty and Ximena Ulloa with BHHS EWM Realty

TOP SALE BUYER AGENTS:

Giulietta Ulloa with BHHS EWM Realty and None with None

BOTTOM SALE LISTING AGENTS:

Itai Tsanaani with Folio360 Group, LLC. and None with None

BOTTOM SALE BUYER AGENTS:

Itai Tsanaani with Folio360 Group, LLC. and None with None




In [57]:
story_string = f'''
\033[1mHED:\033[0m {county_name} condo sales dollar volume {hed_rf} in {previous_month_name} to ${round(df_filtered["PRICE"].sum()/1_000_000)}M 
\033[1mDEK:\033[0m Number of closings {dek_rf} to {len(df_filtered)} from {total_condos_sold_2nd_month} in {month_before_last_name}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county_name} {previous_month_name} Condo Sales Report 
\033[1mSEO DESCRIPTION:\033[0m {county_name}’s condo sales volume {seo} to ${round(df_filtered["PRICE"].sum()/1_000_000)} million in {previous_month_name}.
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m  
\033[1mSocial:\033[0m {county_name} had ${round(df_filtered["PRICE"].sum()/1_000_000):,.0f} million in condo sales in {previous_month_name}, {social} than ${round(int(sales_volume_2nd_month)/1_000_000)} million from {month_before_last_name}, @afarence reports 
\033[1mART:\033[0m 

CHANGE ME

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county_name}, monthly condo sales, condos, condo market, {most_expensive_muni_condo_name.title()}, {most_expensive_muni_location}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m

{county_name}’s NEWS PEG HERE.

{previous_month_name} condo sales totaled ${round(df_filtered["PRICE"].sum()/1_000_000)} million, {nut_graf} from ${int(sales_volume_2nd_month)/1_000_000:.0f} million in {month_before_last_name}. Brokers closed {len(df_filtered)} sales last month, {second_nut_graf_ref} from {total_condos_sold_2nd_month} sales in {month_before_last_name} and XXX in XXX, Multiple Listing Service data from Redfin show.

{previous_month_name} sale prices ranged from ${df_filtered["PRICE"].min():,.0f} to ${df_filtered["PRICE"].max():,.0f} million, with a median sale price of ${df_filtered["PRICE"].median():,.0f}. The price per square foot ranged from ${df_filtered["$/SQUARE FEET"].min():,.0f} to ${df_filtered["$/SQUARE FEET"].max():,.0f}, with a median of ${df_filtered["$/SQUARE FEET"].median():,.0f} per square foot.

In {month_before_last_name}, sales closed with a median price of ${int(median_sales_price_2nd_month):,.0f}, and ${median_psf_2nd_month} per square foot.

A ${df_filtered["PRICE"].max():,.0f} million closing at {most_expensive_muni_condo_name.title()} took the title of priciest sale last month. {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} sold for ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} per square foot after XX days on the market. XXX with XXX had the listing, and XXX with XXX represented the buyer.

TOP SALE LISTING AGENTS:

{top_sale_listing_agent_1} with {top_sale_listing_broker_1} and {top_sale_listing_agent_2} with {top_sale_listing_broker_2}

TOP SALE BUYER AGENTS:

{top_sale_buyer_agent_1} with {top_sale_buyer_broker_1} and {top_sale_buyer_agent_2} with {top_sale_buyer_broker_2}

The sale price was \033[1mmore/less\033[0m than {month_before_last_name}'s priciest sale. Unit XXXX 

In contrast, {previous_month_name}'s cheapest sale was an {least_expensive_muni_condo_name} in {least_expensive_muni_location}, at {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']}. Unit XXX traded for ${df_filtered['PRICE'].min():,.0f} — or ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} per square foot — after XXX days on the market. XXX with XXX handled both sides of the deal.

BOTTOM SALE LISTING AGENTS:

{bottom_sale_listing_agent_1} with {bottom_sale_listing_broker_1} and {bottom_sale_listing_agent_2} with {bottom_sale_listing_broker_2}

BOTTOM SALE BUYER AGENTS:

{bottom_sale_buyer_agent_1} with {bottom_sale_buyer_broker_1} and {bottom_sale_buyer_agent_2} with {bottom_sale_buyer_broker_2}

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="800" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of {previous_month_name}’s notable condo sales:
'''

In [58]:
print(story_string)

print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmax()]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['PRICE'].idxmin()]['buy_broker_name_2']}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['buy_broker_name_2']}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['buy_broker_name_2']}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f} | Listing agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['list_broker_name_2']} | Buyer's agents: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_1']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_1']} and {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_agent_name_2']} with {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['buy_broker_name_2']}")


HED: Miami-Dade County condo sales dollar volume drops in September to $438M 
DEK: Number of closings fell to 626 from 712 in August
FEATURED HED:
SEO HED: Miami-Dade County September Condo Sales Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales volume dropped to $438 million in September.
AUTHOR: Adam Farence
RESEARCH:  
Social: Miami-Dade County had $438 million in condo sales in September, lower than $470 million from August, @afarence reports 
ART: 

CHANGE ME

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, monthly condo sales, condos, condo market, Oceana Key Biscayne Condo, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:

Miami-Dade County’s NEWS PEG HERE.

September condo sales totaled $438 million, falling from $470 million in August. Brokers closed 626 sales last month, down from 712 sales in August and XXX in XXX

# Print links for notable sales

## Top sale

In [65]:
print(df_filtered.loc[df_filtered['PRICE'].idxmax()]['URL'])

https://www.redfin.com/FL/Key-Biscayne/360-Ocean-Dr-33149/unit-1002S/home/79133323


## Cheapest sale

In [66]:
print(df_filtered.loc[df_filtered['PRICE'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami/15201-NE-6th-Ave-33162/unit-C210/home/43356954


## Highest PSF

In [67]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['URL'])

https://www.redfin.com/FL/Surfside/9111-Collins-Ave-33154/unit-N713/home/79133163


## Lowest PSF

In [68]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/Doral/5300-NW-87th-Ave-33178/unit-512/home/43402972


## Newest

In [69]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['URL'])

https://www.redfin.com/FL/Miami/2655-S-Bayshore-Dr-33133/unit-1501/home/191480913


## Oldest

In [70]:
print(df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['URL'])

https://www.redfin.com/FL/Miami-Beach/752-Meridian-Ave-33139/unit-19/home/42802202


## Time on Market Calculator

In [72]:
from datetime import datetime, timedelta

################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 7, 18) ## List (Earlier) date
date2 = datetime(2024, 9, 12) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

56


## Access second highest (or whatever is needed)

In [58]:
# ### Second highest ###
# # Sort the DataFrame by '$/SQUARE FEET' in descending order
# second_highest = df_filtered.sort_values(by='$/SQUARE FEET', ascending=False)

# # Get the URL of the entry with the second highest '$/SQUARE FEET' value
# second_highest_url = second_highest.iloc[1]['URL']  # `.iloc[1]` accesses the second row

# print(second_highest_url)